In [1]:
import sys
sys.path += [
    './lib/',
    './lib/field_mappers'
]

In [2]:
import time
from pymongo import MongoClient
import spacy

from extract_biographies import extract_biographies
from generate_sample_collection import copy_sample
from extract_first_sentence import FirstSentenceExtractor
from vectorizer import Vectorizer
from pageview_aggregator import PageviewAggregator
from query import Searcher

In [3]:
SPACY_MODEL = 'en_vectors_web_lg'

In [5]:
db = MongoClient().who
# clean up test db
# db.bios.remove({})
# db.sample_bios.remove({})

In [5]:
extract_biographies(collection_name='bios', limit=100_000)

inserting batch 0
inserting batch 1
inserting batch 2
inserting batch 3
inserting batch 4
inserting batch 5
inserting batch 6
inserting batch 7
inserting batch 8
inserting batch 9
inserting batch 10


In [7]:
db.sample_bios.count_documents({})

10000

In [4]:
copy_sample('bios', 'sample_bios', 10_000)
db.sample_bios.count_documents({})

NameError: name 'db' is not defined

In [8]:
FirstSentenceExtractor().extract('sample_bios')

insertation count: 0
insertation count: 1000
insertation count: 2000
insertation count: 3000
insertation count: 4000
insertation count: 5000
insertation count: 6000
insertation count: 7000
insertation count: 8000
insertation count: 9000


In [9]:
next(db.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence'])

In [10]:
next(db.sample_bios.find({}))['first_sentence']

'Vilfredo Federico Damaso Pareto  was an Italian engineer, sociologist, economist, political scientist, and philosopher.'

In [11]:
nlp = spacy.load(SPACY_MODEL)

In [12]:
Vectorizer(nlp).vectorize_text(
    collection_name='sample_bios',
    text_field_name='first_sentence',
    vector_field_name='fresh_new_vectorization'
)

insertation count: 0
insertation count: 1000
insertation count: 2000
insertation count: 3000
insertation count: 4000
insertation count: 5000
insertation count: 6000
insertation count: 7000
insertation count: 8000
insertation count: 9000


In [13]:
pva = PageviewAggregator()
# do but once:
# pva.aggregate('./data/monthly_view_totals/filtered')
pva.load_totals()
pva.insert(collection_name='sample_bios')

pageview total insertation count: 0


In [14]:
next(db.sample_bios.find({}))['fresh_new_vectorization'][:9]

[-0.02991873398423195,
 0.17837101221084595,
 0.06707080453634262,
 0.004216194152832031,
 0.04910694807767868,
 0.22781610488891602,
 -0.042344801127910614,
 -0.20524628460407257,
 -0.1988140046596527]

In [15]:
next(db.sample_bios.find({})).keys()

dict_keys(['_id', 'name', 'bio', 'first_sentence', 'fresh_new_vectorization', 'views'])

In [65]:
%run './lib/query'

In [66]:
start = time.time()
result = Searcher(
    nlp,
    collection_name='sample_bios',
    vector_field_name='fresh_new_vectorization'
).query('female chemist who died from radiation poisoning', limit=10)
end = time.time()
print(end - start)
result

1.157724142074585


[{'name': 'Friedrich Ernst Dorn',
  'cosine_distance': 0.7749917213552745,
  'views': 364,
  'prominence': 1.0118641348334851,
  'rank': 0.7841863276322683},
 {'name': 'Marie Curie',
  'cosine_distance': 0.7615277365570745,
  'views': 145626,
  'prominence': 1.0240625348483006,
  'rank': 0.7798520242559266},
 {'name': 'Edward Jenner',
  'cosine_distance': 0.7476269055228573,
  'views': 26533,
  'prominence': 1.0205812205130782,
  'rank': 0.7630139797269335},
 {'name': 'Wilhelm Röntgen',
  'cosine_distance': 0.7435304087465205,
  'views': 20356,
  'prominence': 1.020040428173996,
  'rank': 0.7584310764981872},
 {'name': 'W. H. R. Rivers',
  'cosine_distance': 0.7448903312880188,
  'views': 2161,
  'prominence': 1.0154751720136845,
  'rank': 0.7564176372960314},
 {'name': 'Billy the Kid',
  'cosine_distance': 0.7328468215456974,
  'views': 70120,
  'prominence': 1.022566789573722,
  'rank': 0.7493848215572901},
 {'name': 'Johan August Arfwedson',
  'cosine_distance': 0.7392873760957566,


In [ ]:
VectorNormalizer().normalize('sample_bios')
searcher = Searcher(collection_name='bios')
searcher.query('fashion entrepreneur', 8)

In [ ]:
normal_7e22009